<a href="https://colab.research.google.com/github/Rhcsky/Colab-pytorch/blob/master/Logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install konlpy
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchtext.data import Field, TabularDataset, Iterator

import numpy as np
import pandas
import copy
import timeit
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
# from konlpy.tag import Hannanum, Kkma, Okt, Mecab

# tagger = Okt()
# tokenize = tagger.morphs

if torch.cuda.is_available():
  print("You can use cuda GPU")
  device = "cuda"
else:
  device = "cpu"

You can use cuda GPU


In [0]:
start = timeit.default_timer()

ID = Field(sequential=False,use_vocab=False,batch_first=True)
TEXT = Field(use_vocab=True,include_lengths=False,batch_first=True, stop_words=['XXX','.'])
LABEL = Field(sequential=False,use_vocab=False, batch_first = True, preprocessing = lambda x : float(x))

data_fields = [('id',ID),('year_month',None),('text',TEXT),('smishing',LABEL)]

train = TabularDataset("/content/drive/My Drive/Colab Notebooks/data/mecab_train_split.csv",format='csv',fields=data_fields, skip_header=True)
# train_split tokenize_train
print(train.examples[0].id)
print(train.examples[0].text)
print(train.examples[0].smishing)
stop = timeit.default_timer()

print(stop-start)


25370
['은JX', '행시NNG', '화XSN', '점NNG', '팀장NNG', '전화NNG', ':SC', '-SY', '-SY', '팩스NNG', ':SC', '-SY', '-SY', '입니다VCP+EF', '!SF', '!SF']
0.0
10.103664409999965


In [0]:
TEXT.build_vocab(train,min_freq = 10)
len(TEXT.vocab)

13704

In [0]:
val = TabularDataset("/content/drive/My Drive/Colab Notebooks/data/mecab_val_split.csv",format='csv',fields=data_fields, skip_header=True)
train_loader = Iterator(train, batch_size=128, device=device, repeat=False, sort_key=lambda x: len(x.text),sort_within_batch=True)
val_loader = Iterator(val, batch_size=512, device=device, repeat=False, sort_key=lambda x: len(x.text),sort_within_batch=True,shuffle=False)

In [0]:
class LogisticClassifier(nn.Module):
  def __init__(self,input,embed_dim,output):
    super(LogisticClassifier,self).__init__()

    self.embed = nn.Embedding(input,embed_dim)

    self.linear = nn.Linear(embed_dim, output)
    
  def forward(self,x):
    x = self.embed(x)
    y = self.linear(x.mean(1))
    y = torch.sigmoid(y)

    return y

In [0]:
model = LogisticClassifier(len(TEXT.vocab),16,1)
model = model.to('cuda')
loss_function = nn.BCELoss().to(device) # BCE는 마지막에 sigmoid
# loss_function = nn.CrossEntropyLoss().to(device) #CE는 마지막에 linear
optimizer = optim.SGD(model.parameters(),lr=0.05)
# optimizer = SGD, Adam
print(model)

LogisticClassifier(
  (embed): Embedding(13704, 16)
  (linear): Linear(in_features=16, out_features=1, bias=True)
)


In [0]:
EPOCH = 10
best_error = 100

for epoch in tqdm_notebook(range(EPOCH),desc='EPOCH'):
  model.train()
  pred_list = []
  target_list = []
  losses = 0
  for i,batch in enumerate(tqdm_notebook(train_loader,desc = "Train")):
      optimizer.zero_grad()
      # batch.text = batch.text.float()
      preds = model(batch.text).squeeze()
      batch.smishing = batch.smishing.float()
      loss = loss_function(preds,batch.smishing)
      loss.backward()
      optimizer.step()
      #save preds
      losses += loss.item()
      for idx in range(len(preds)):
        pred_list.append(preds[idx].item())
        target_list.append(batch.smishing[idx].item())

  #model scoring with ACC
  pred_numpy = np.asarray(pred_list)
  target_numpy = np.asarray(target_list)
  modeling_score = roc_auc_score(target_numpy,pred_numpy)
  print(pred_numpy,target_numpy)
  for idx, a in enumerate(pred_numpy):
    if a>0.5:
      pred_numpy[idx] = 1
    else:
      pred_numpy[idx] = 0
  res = (pred_numpy == target_numpy).sum()
  print(f'TRAIN score = {modeling_score} || loss = {losses} || ACC = {len(pred_list)-res}')
  

  model.eval()
  pred_list=[]
  target_list = []
  losses = 0
  for i, batch in enumerate(tqdm_notebook(val_loader,desc="Val")): 
      preds = model(batch.text).squeeze()
      batch.smishing = batch.smishing.float()
      loss = loss_function(preds,batch.smishing)
      losses += loss.item()
      # preds = torch.softmax(preds,dim=1)
      for idx in range(len(preds)):
        pred_list.append(preds[idx].item())
        target_list.append(batch.smishing[idx].item())

  #model scoring with AUC
  pred_numpy = np.asarray(pred_list)
  target_numpy = np.asarray(target_list)
  score = roc_auc_score(target_numpy,pred_numpy)
  print(pred_numpy,target_numpy)
  for idx, a in enumerate(pred_numpy):
    if a>0.5:
      pred_numpy[idx] = 1
    else:
      pred_numpy[idx] = 0
  error = len(pred_list) - (pred_numpy == target_numpy).sum()
  print(f'EVAL score = {score} || loss = {losses} || Error = {error}')



  if best_error > error:
    best_error = error
    best_model_wts = copy.deepcopy(model.state_dict())
    print("Best model is saved!!")
  if error == 0:
    print("Optimization has been completed and will be terminated in advance.")
    break

model.load_state_dict(best_model_wts)

[0.96952093 0.90186113 0.869744   ... 0.00222479 0.0021779  0.00211065] [1. 1. 1. ... 0. 0. 0.]
TRAIN score = 0.9880740136461676 || loss = 114.64247023686767 || ACC = 6126


[0.95073277 0.9461351  0.90555847 ... 0.00209623 0.00209213 0.0020317 ] [1. 1. 1. ... 0. 0. 0.]
EVAL score = 0.9888647311540004 || loss = 12.437299694865942 || Error = 2422


[0.9657625  0.94021398 0.88870919 ... 0.00248791 0.00237724 0.0023453 ] [1. 1. 1. ... 0. 0. 0.]
TRAIN score = 0.988214178564346 || loss = 113.81513070128858 || ACC = 6093


[0.95363873 0.95042479 0.91202509 ... 0.00245739 0.00245707 0.0023837 ] [1. 1. 1. ... 0. 0. 0.]
EVAL score = 0.9890052738134157 || loss = 12.263214200735092 || Error = 2518


[0.97087121 0.96070629 0.93739301 ... 0.00204402 0.00206171 0.00202795] [1. 1. 1. ... 0. 0. 0.]
TRAIN score = 0.9883197467997211 || loss = 113.1972581371665 || ACC = 6149


...

[0.97309554 0.96532869 0.95188874 ... 0.00219341 0.00246007 0.00220554] [1. 1. 1. ... 0. 0. 0.]
TRAIN score = 0.9889854423542281 || loss = 109.77698881272227 || ACC = 6016


[0.95646662 0.95723236 0.91679889 ... 0.00180555 0.00181803 0.00175311] [1. 1. 1. ... 0. 0. 0.]
EVAL score = 0.9896754620557212 || loss = 11.906471367925406 || Error = 2354


[0.96689677 0.9693712  0.94463944 ... 0.00201187 0.00188823 0.00189118] [1. 1. 1. ... 0. 0. 0.]
TRAIN score = 0.9891172004564617 || loss = 109.27767535019666 || ACC = 5954


[0.95660478 0.95846659 0.9179728  ... 0.00192577 0.00194246 0.00187109] [1. 1. 1. ... 0. 0. 0.]
EVAL score = 0.9898405224004817 || loss = 11.779294330626726 || Error = 2344


KeyboardInterrupt: ignored

In [0]:
model.load_state_dict(best_model_wts)
for idx, a in enumerate(pred_numpy):
  if a>0.5:
    pred_numpy[idx] = 1
  else:
    pred_numpy[idx] = 0

res = pred_numpy == target_numpy

cnt = 0
for a in res:
  if a == True:
    cnt += 1


print(cnt/len(res), cnt, len(res), len(res)-cnt)

In [0]:
test_fields = [('id',ID),('year',None),('text',TEXT)]
test = TabularDataset("/content/drive/My Drive/Colab Notebooks/data/mecab_public_test.csv",format='csv',fields=test_fields, skip_header=True)
# tokenize_public_test val_split
test_loader = Iterator(test, batch_size=1000, device=device, repeat=False, shuffle=False)

In [0]:
ids = []
labels = []
pred_list = []

model.eval()
for i, batch in enumerate(tqdm_notebook(test_loader)):
    pred = model(batch.text)
    pred = torch.softmax(pred,dim=1)
    for idx in range(len(pred)):
      ids.append(batch.id[idx].item())
      pred_list.append(pred[idx][1].item())
      labels.append(pred[idx].max(0)[1].item())

print(pred_list)


result_data = pandas.DataFrame({'id':ids,'smishing':pred_list})
result_dataL = pandas.DataFrame({'id':ids,'smishing':pred_list, 'label':labels})
print(result_data)
result_data.to_csv('/content/drive/My Drive/Colab Notebooks/data/result_data.csv',index=False)

In [0]:
cnt = 0
for a in labels:
  if a == 1:
    cnt += 1
print(cnt)

In [0]:
eouchan = pandas.read_csv("/content/drive/My Drive/Colab Notebooks/data/submission.csv")

smishing = 0
for a in eouchan.label:
  if a == 1:
    smishing = smishing+1

print(smishing)


you = []
my = []
co = []

for e,m,id in zip(eouchan.label,labels,ids):
  if e == m:
    co.append(id)
  elif e>m:
    you.append(id)
  else:
    my.append(id)

print(f'my = {len(my)}\n{my}')
print(f'you = {len(you)}\n{you}')
print(f'co = {len(co)}\n{co}')

print(len(my)/len(ids))

In [0]:
Path = '/content/drive/My Drive/Colab Notebooks/data/model/model.pt'
Path2 = '/content/drive/My Drive/Colab Notebooks/data/model/remodel.pt'
Path3 = '/content/drive/My Drive/Colab Notebooks/data/model/allmodel.pt'

state = {
    'epoch' : EPOCH,
    'state_dict' : model.state_dict(),
    'optimizer' : optimizer.state_dict(),
    'loss' : loss_function
}
torch.save(model.state_dict(),Path)
torch.save(state,Path2)
torch.save(model,Path3)